# A/B Testing Project

## Contents  <a name="back"></a>

1. [Introduction](#introduction)
2. [Part 1: Prioritizing Hypotheses](#hypotheses)
   1. [Data loading and inspection](#data1)
   2. [ICE](#ice)
   3. [RICE](#rice)
3. [Part 2. A/B Test Analysis](#ab_test)

## Introduction <a name="introduction"></a>

This proyect is for the A/B Testing sprint on Tripleten's Data Analyst course. It consists on two parts:

- Part 1 - Prioritizing hypotheses: We have a list of hypotheses that we need to prioritize according to ICE and RICE, and draw conclusions.
- Part 2 - A/B Test Analysis: For this part we have data on visitors and orders. Our objective is to determine the results of the test.

For this project we'll be using the following:
- Python 3.9.5
- Pandas 1.2.4
- Matplotlib.pyplot 3.3.4
- Numpy 1.20.3
- Scipy.stats 1.7.3

Versions were chosen so they match as closely as possible the versions available on the Tripleten servers

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import datetime as dt

## Part 1: Prioritizing Hypotheses <a name="hypotheses"></a>

### Data loading and inspection <a name="data1"></a>

We have a table with the current hypotheses that are awaiting testing. We'll have to load several tables for this proyect, so lets create a function to acomplish that.

In [2]:
def load_csv(path: str, sep: str):
    try: # Local path
        filepath = f'datasets/{path}'
        df = pd.read_csv(filepath, sep=sep)
    except: # Tripleten server path
        filepath = f'/datasets/{path}'
        df = pd.read_csv(filepath, sep=sep)
    return df

Now we need to load the table and see what it contains. This is a short table.

In [3]:
# Load the table
hypotheses = load_csv('hypotheses_us.csv', ';')

In [4]:
hypotheses.columns = [x.lower().replace(' ', '_') for x in hypotheses.columns]

In [5]:
hypotheses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   hypothesis  9 non-null      object
 1   reach       9 non-null      int64 
 2   impact      9 non-null      int64 
 3   confidence  9 non-null      int64 
 4   effort      9 non-null      int64 
dtypes: int64(4), object(1)
memory usage: 488.0+ bytes


There is only 9 hypotheses. Lets see them.

In [6]:
hypotheses

,hypothesis,reach,impact,confidence,effort
0,Add two new channels for attracting traffic. T...,3,10,8,6
1,Launch your own delivery service. This will sh...,2,5,4,10
2,Add product recommendation blocks to the store...,8,3,7,3
3,Change the category structure. This will incre...,8,3,3,8
4,Change the background color on the main page. ...,3,1,1,1
5,Add a customer review page. This will increase...,3,2,2,3
6,Show banners with current offers and sales on ...,5,3,8,3
7,Add a subscription form to all the main pages....,10,7,8,5
8,Launch a promotion that gives users discounts ...,1,9,9,5


There isn't any preprocessing to be done with this. It's ready to be used.

[Back to Contents](#back)

### ICE <a name="ice"></a>

The ICE score is calculated like this: `(Impact * Confidence) / Effort`. Lets add an ICE column to the data.

In [7]:
hypotheses['ICE'] = hypotheses['impact'] * hypotheses['confidence'] / hypotheses['effort']

In [8]:
hypotheses.sort_values(by='ICE', ascending=False)

,hypothesis,reach,impact,confidence,effort,ICE
8,Launch a promotion that gives users discounts ...,1,9,9,5,16.200000
0,Add two new channels for attracting traffic. T...,3,10,8,6,13.333333
7,Add a subscription form to all the main pages....,10,7,8,5,11.200000
6,Show banners with current offers and sales on ...,5,3,8,3,8.000000
2,Add product recommendation blocks to the store...,8,3,7,3,7.000000
1,Launch your own delivery service. This will sh...,2,5,4,10,2.000000
5,Add a customer review page. This will increase...,3,2,2,3,1.333333
3,Change the category structure. This will incre...,8,3,3,8,1.125000
4,Change the background color on the main page. ...,3,1,1,1,1.000000


The ICE score is placing hypotheses 8, 0 and 7 in the top place. These are the following:

- 8: Launch a promotion that gives users discounts on their birthdays
- 0: Add two new channels for attracting traffic. This will bring 30% more users
- 7: Add a subscription form to all the main pages. This will help you compile a mailing list

These all have top scores in Impact and Confidence, with a medium Effort score.

Lets see how that changes with RICE.

[Back to Contents](#back)

### RICE <a name="rice"></a>

Similar to ICE, RICE will give us a score that will let us prioritize our hypotheses. This time we add Reach to the equation:

`(Reach * Impact * Confidence) / Effort`

Lets calculate it and add it to the table.

In [9]:
hypotheses['RICE'] = (hypotheses['reach'] * hypotheses['impact'] * hypotheses['confidence']) / hypotheses['effort']

In [10]:
hypotheses.sort_values(by='RICE', ascending=False)

,hypothesis,reach,impact,confidence,effort,ICE,RICE
7,Add a subscription form to all the main pages....,10,7,8,5,11.200000,112.0
2,Add product recommendation blocks to the store...,8,3,7,3,7.000000,56.0
0,Add two new channels for attracting traffic. T...,3,10,8,6,13.333333,40.0
6,Show banners with current offers and sales on ...,5,3,8,3,8.000000,40.0
8,Launch a promotion that gives users discounts ...,1,9,9,5,16.200000,16.2
3,Change the category structure. This will incre...,8,3,3,8,1.125000,9.0
1,Launch your own delivery service. This will sh...,2,5,4,10,2.000000,4.0
5,Add a customer review page. This will increase...,3,2,2,3,1.333333,4.0
4,Change the background color on the main page. ...,3,1,1,1,1.000000,3.0


The RICE score also puts hypotheses 7 and 0 in the top 3. 

- Hypotheses 7 already had a high ICE score. Having maximum reach bumped it's position to the first place, by far. It has more than double the score of the next best.
- Hypotheses 2 was in the middle of the pack according to ICE. It has a good Reach score, which helped it move up. But the main reason it got to second place is that the rest of the ICE high scorers have a pretty low Reach, drastically impacting their RICE score.
- Hypotheses 8 was knocked down several positions due to its low reach.
- Hypotheses 0 had a great ICE score, and with that it managed to stay in the top 3 even tough it has a low Reach score, mostly because that is the case with most of the hypotheses.

Overall, hypotheses 7 seems to be the most relevant right now.

[Back to Contents](#back)

## Part 2: A/B Test Analysis <a name="ab_test"></a>

For this part we have the data of the A/B test. We will be analizing the results in order to conclude if the tested hypotheses show an improvement or not. 

### Data loading and inspection <a name="data2"></a>

We have two tables to work with here. Visitors and Orders. Let's get them ready.

In [11]:
orders = load_csv('orders_us.csv', ',')
visits = load_csv('visits_us.csv', ',')

In [18]:
orders.info()
orders.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   transactionId  1197 non-null   int64         
 1   visitorId      1197 non-null   int64         
 2   date           1197 non-null   datetime64[ns]
 3   revenue        1197 non-null   float64       
 4   group          1197 non-null   category      
dtypes: category(1), datetime64[ns](1), float64(1), int64(2)
memory usage: 38.8 KB


,transactionId,visitorId,date,revenue,group
0,3667963787,3312258926,2019-08-15,30.4,B
1,2804400009,3642806036,2019-08-15,15.2,B
2,2961555356,4069496402,2019-08-15,10.2,A
3,3797467345,1196621759,2019-08-15,155.1,B
4,2282983706,2322279887,2019-08-15,40.5,B


It has no null values, and the column names are correctly formatted. We need to set the proper types for the date and group.

In [13]:
# Set date and group to their correct type
orders['date'] = orders['date'].map(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))
orders['group'] = orders['group'].astype('category')

Now lets do the same for visits.

In [19]:
visits.info()
visits.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    62 non-null     object
 1   group   62 non-null     object
 2   visits  62 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 1.6+ KB


,date,group,visits
0,2019-08-01,A,719
1,2019-08-02,A,619
2,2019-08-03,A,507
3,2019-08-04,A,717
4,2019-08-05,A,756


There is no null values. Column names are ok. We only need to correct the types.

In [20]:
visits['date'] = visits['date'].map(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))
visits['group'] = visits['group'].astype('category')

Now lets check some possible errors in the data:

Do we have repeated `date-group` combinations in `visits`?

In [23]:
# Check for rows with the same date-group combination. A sum of 0 mean there are no duplicates
visits[['date', 'group']].duplicated().sum()

0

Do we have duplicated `transactionId`'s?

In [24]:
# There are 1197 rows in the orders df. We should get the same value with nunique if there isn't any duplicates
orders['transactionId'].nunique()

1197

Do the `date` range match in `orders` and `visits`?

In [34]:
# The min and max values should match
print('Orders date range:')
print(orders['date'].min())
print(orders['date'].max())
print('---')
print('Visits date range:')
print(visits['date'].min())
print(visits['date'].max())

Orders date range:
2019-08-01 00:00:00
2019-08-31 00:00:00
---
Visits date range:
2019-08-01 00:00:00
2019-08-31 00:00:00


Is there a value other than A and B in the group columns?

In [48]:
print(orders['group'].unique())
print(visits['group'].unique())

['B', 'A']
Categories (2, object): ['B', 'A']
['A', 'B']
Categories (2, object): ['A', 'B']


Does any `visitorId` have orders on both groups?

In [41]:
double_groups = orders.groupby('visitorId')['group'].nunique().sort_values(ascending=False).reset_index().query('group > 1')
double_groups.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58 entries, 0 to 57
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   visitorId  58 non-null     int64
 1   group      58 non-null     int64
dtypes: int64(2)
memory usage: 1.4 KB


There are 58 visitors that were part of both groups. We should drop them from the test. It is possible to drop their orders, but we have no way of droping their visits, since we only have pre-agregated data.

In [44]:
orders = orders[np.logical_not(orders['visitorId'].isin(double_groups['visitorId']))]

In [45]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1016 entries, 0 to 1196
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   transactionId  1016 non-null   int64         
 1   visitorId      1016 non-null   int64         
 2   date           1016 non-null   datetime64[ns]
 3   revenue        1016 non-null   float64       
 4   group          1016 non-null   category      
dtypes: category(1), datetime64[ns](1), float64(1), int64(2)
memory usage: 40.8 KB


The data is ready for work. Let's start.

[Back to contents](#back)